In [21]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from tqdm import tqdm
import re

import pickle as pkl

In [2]:
NnL = pd.read_excel('1.Dataset_files/CharacterData.xlsx',index_col=0)

In [20]:
edge_aliases = {'Vernon Dursley': 'Mr. Dursley', 'Petunia Dursley': 'Mrs. Dursley', 'Lily Potter': 'Mrs. Potter', 'Harry Potter': 'Mr. Potter', 'Lord Voldemort': 'My Lord',
                'Albus Dumbledore': 'Dumbledore', 'Minerva McGonagall': 'McGonagall', 'Dedalus Diggle': 'Diggle', 'Poppy Pomfrey': 'Pomfrey',
                'Rubeus Hagrid': 'Hagrid', 'Arabella Figg': 'Mrs. Figg', 'Mr Paws': 'Mr. Paws', 'A boa constrictor': 'Boa Constrictor',
                'Mr Evans': 'Mr. Evans', 'Mrs Evans': 'Mrs. Evans', 'Quirinus Quirrell': 'Quirrell', 'Draco Malfoy': 'Malfoy',
                'Garrick Ollivander': 'Mr. Ollivander', 'Molly Weasley': 'Mrs. Weasley', 'Ron Weasley': 'Mr. Weasley',
                'Neville Longbottom': 'Longbottom', 'Cornelius Agrippa': 'Agrippa', 'Claudius Ptolemy': 'Ptolemy', 'Gellert Grindelwald': 'Grindelwald',
                'Hermione Granger': 'Miss Granger', 'Vincent Crabbe': 'Crabbe', 'Gregory Goyle': 'Goyle', 'Fat Friar': 'Friar',
                'Nearly-Headless Nick': 'Nearly Headless Nick', 'Mr Finnigan': 'Mr. Finnigan', 'Mrs Finnigan': 'Mrs. Finnigan', 
                'Severus Snape': 'Snape', 'Argus Filch': 'Filch', 'Rolanda Hooch': 'Madam Hooch', 'Mrs Norris': 'Mrs. Norris', 
                'Pomona Sprout': 'Professor Sprout', 'Cuthbert Binns': 'Professor Binns', 'Filius Flitwick': 'Professor Flitwick', 
                'Oliver Wood': 'Wood', 'Marcus Flint': 'Flint', 'Alicia Spinnet': 'Spinnet', 'Adrian Pucey': 'Pucey', 
                'Terence Higgs': 'Higgs', 'Irma Pince': 'Madam Pince', 'Hogwarts Giant Squid': 'giant squid', 'Mr Mason': 'Mr. Mason',
                'Mrs Mason': 'Mrs. Mason', 'Gilderoy Lockhart': 'Lockhart', 'Mr Borgin': 'Mr. Borgin', 'Mr Granger': 'Mr. Granger',
                'Mrs Granger': 'Mrs. Granger', 'Dr Filibuster': 'Dr. Filibuster',
                'Colin Creevey': 'Mr. Creevey', 'Sir Patrick Delaney-Podmore': 'Sir Patrick', 'D.J. Prod': 'D. J. Prod', 'Mrs Skower': 'Mrs. Skower',
                'Ernie Macmillan': 'Macmillan', 'Aurora Sinistra': 'Professor Sinistra', 'Armando Dippet': 'Dippet', 'Hogwarts Board of Governors': 'governors', 'Tom Riddle Senior': 'Tom',
                'Remus Lupin': 'Lupin', 'Silvanus Kettleburn': 'Professor Kettleburn', 'Sybill Trelawney': 'Professor Trelawney',
                'Septima Vector': 'Professor Vector', 'Roger Davies': 'Davies', 'Cassius Warrington': 'Warrington', 'Graham Montague': 'Montague',
                'Peregrine Derrick': 'Derrick', 'Lucian Bole': 'Bole', 'Walden Macnair': 'Macnair', 'Mary Riddle': 'Mrs. Riddle', 'Thomas Riddle': 'Mr. Riddle',
                'Viktor Krum': 'Krum', 'Amos Diggory': 'Mr. Diggory', 'Mr Roberts': 'Mr. Roberts', 'Mr Payne': 'Mr. Payne', 
                'Mr Wood': 'Mr. Wood', 'Mrs Wood': 'Mrs. Wood', 'Broderick Bode': 'Bode', 'Saul Croaker': 'Croaker', 'Roddy Pontner': 'Pontner',
                'Vasily Dimitrov': 'Dimitrov', 'Clara Ivanova': 'Ivanova', 'Lev Zograf': 'Zograf', 'Alexei Levski': 'Levski', 'Pyotr Vulchanov': 'Vulchanov',
                'Ivan Volkov': 'Volkov', 'Barry Ryan': 'Ryan', 'Aidan Lynch': 'Lynch', 'Hassan Mostafa': 'Mostafa', 'Mrs Roberts': 'Mrs. Roberts',
                'Olympe Maxime': 'Madam Maxime', 'Rita Skeeter': 'Skeeter', 'Alastor Moody': 'Mad-Eye', 'Barty Crouch Jr': 'Master Barty',
                'Igor Karkaroff': 'Karkaroff', 'Mykew Gregorovitch': 'Gregorovitch', 'Wilhelmina Grubbly-Plank': 'Professor Grubbly-Plank', 'Evan Rosier': 'Rosier',
                'Mrs Fudge': 'Mrs. Fudge', 'Antonin Dolohov': 'Dolohov', 'Augustus Rookwood': 'Rookwood', 'Alice Longbottom': "Frank Longbottom's wife",
                'Mrs Diggory': 'Mrs. Diggory', 'Mr Krum': 'Mr. Krum', 'Mrs Krum': 'Mrs. Krum', 'Mr Prentice': 'Mr. Prentice',
                'Kingsley Shacklebolt': 'Shacklebolt', 'Sturgis Podmore': 'Podmore', 'Rufus Scrimgeour': 'Scrimgeour', 'Dolores Umbridge': 'Umbridge',
                'Wilbert Slinkhard': 'Slinkhard', 'Xenophilius Lovegood': 'Mr. Lovegood', 'Mrs Chang': 'Mrs. Chang', 'Barnabus the Barmy': 'Barnabas the Barmy', 'Mr Edgecombe': 'Mr. Edgecombe',
                'Mr Chang': 'Mr. Chang', 'Andrew Kirke': 'Kirke', 'Jack Sloper': 'Slope', 'Dexter Fortescue': 'Fortescu',
                'Hippocrates Smethwyck': 'Healer Smethwyck', 'Augustus Pye': 'Pye', 'Miriam Strout': 'Healer Strout', 'John Dawlish': 'Dawlish',
                'Mr Montague': 'Mr. Montague', 'Mrs Montague': 'Mrs. Montague', 'Eddie Carmichael': 'Carmichael', 'Harold Dingle': 'Dingle',
                'Dr Ubbly': 'Dr. Ubbly', 'Mrs Goyle': 'Mrs. Goyle', 'Corban Yaxley': 'Yaxley', 'Amycus Carrow': 'Carrows', 'Alecto Carrow': 'Carrows',
                'Fenrir Greyback': 'Greyback', 'Horace Slughorn': 'Slughorn', 'Cormac McLaggen': 'McLaggen', 'Marcus Belby': 'Belby',
                'Mr Belby': 'Mr. Belby', 'Bob Ogden': 'Ogden', 'Marvolo Gaunt': 'Mr. Gaunt', 'Mr Patil': 'Mr. Patil',
                'Mrs Patil': 'Mrs. Patil', 'Mr Midgen': 'Mr. Midgen', 'Mrs Abbott': 'Mrs. Abbott', 'Demelza Robins': 'Robins',
                'Jimmy Peakes': 'Peakes', 'Ritchie Coote': 'Coote', 'Melinda Bobbin': 'Bobbin', 'Caractacus Burke': 'Burkes',
                'Mrs Cole': 'Mrs. Cole', 'Billy Stubbs': 'Stubbs', 'Eric Whalley': 'Whalley', 'Amy Benson': 'Benson', 'Dennis Bishop': 'Bishop',
                'Galatea Merrythought': 'Professor Merrythought', 'Wilkie Twycross': 'Twycross', 'Rosalind Antigone Bungs': 'Bungs', 'Pius Thicknesse': 'Thicknesse',
                'Charity Burbage': 'Professor Burbage', 'Dragomir Gorgovitch': 'Gorgovitch', 'Reginald Cattermole': 'Cattermole', 'Albert Runcorn': 'Runcorn',
                'Ignotus Peverell': 'youngest brother', 'Antioch Peverell': 'oldest brother'}

In [54]:
def get_edge_alias(character: str = 'Harry Potter', alias_list: list = []):
    with open("Temp/character_aliases_edgecases.pkl", "rb") as f:
        edge_aliases = pkl.load(f)

    if character in edge_aliases.keys():
        alias_list.append(edge_aliases[character])
    
    return alias_list

def add_first_name(character: str = 'Harry Potter', alias_list: list = []):
    none_aliases = ['A', 'Mr', 'Mrs', 'Dr', 'Manager', 'The', 'Father','Sorting', 'wizard', 'Chancellor', 'Workmen', 'Waitress', 'Sir',
                    'Nearly-Headless', 'Fat','Aged', 'Blood-Sucking', 'Forbidden','Unidentified', 'Zoo', 'Kepper', 'Muggle', 'Muggle-Born',
                    'Senior', 'Junior', 'Board', 'Committee','Academy', 'Ministry', 'Department',
                    'Little', 'Great', 'Old', 'Young', 'Head', 'Headmaster', 'Headmistress','Weird','Care', 'Montgomery', 'Hogwarts', 'Frank']
    
    none_name = ["'s", "s'", "family"]

    if ((character.split(' ')[0] not in none_aliases) and (all([s not in character for s in none_name])) and (character.split(' ')[0].isalpha())):
        alias_list.append(character.split(' ')[0])
    
    return alias_list

def get_aliases(character: str = 'Harry Potter', link: str = '/wiki/Harry_Potter', df: pd.DataFrame = NnL):
    query = requests.get(f"https://harrypotter.fandom.com/api.php?action=parse&page={link.split('/')[-1].split('#')[0]}&format=json").json()
    HTML = query['parse']['text']['*']
    soup = BeautifulSoup(HTML, 'html.parser')

    # Get aliases
    try:
        # Character has aliases
        alias_ = soup.find(string='Also known as').findNext("div")
        if alias_.find_all('li') != []:
            # Character has multiple aliases
            alias_ = [i.get_text() for i in alias_.find_all('li')]
            alias_ = [re.split('\s?(\()|(\[)',j)[0] for j in alias_]
        else:
            # Character has only one alias
            alias_ = [re.split('\s?(\()|(\[)',alias_.get_text())[0]]
    except:
        # Character has no aliases
        alias_ = []

    # Ensure that aliases are not the same as a character name
    alias_ = [i for i in alias_ if not df['Name'].str.contains(f'{i}', regex=False).any()]

    # Add edge cases and first name
    alias_ = get_edge_alias(character = character, alias_list = alias_)
    alias_ = add_first_name(character = character, alias_list = alias_)

    # Ensure that aliases are unique and not the same as the character's name
    alias_ = list(set(alias_))
    alias_ = [i for i in alias_ if i != character]

    return alias_

In [58]:
aliases = [get_aliases(character = i, link = j, df = NnL) for i,j in tqdm(zip(NnL['Name'], NnL['Link']))]
NnL['Aliases'] = aliases

707it [04:48,  2.45it/s]


In [57]:
NnL

,Name,Link,Aliases
0,Vernon Dursley,/wiki/Vernon_Dursley,"[Mr. Dursley, Vernon]"
1,Petunia Dursley,/wiki/Petunia_Dursley,"[Tuney, Petunia, Mrs. Dursley]"
2,Dudley Dursley,/wiki/Dudley_Dursley,"[Dudley, Dudders, Diddykins, Popkin, Big 'D', ..."
3,Lily Potter,/wiki/Lily_J._Potter,"[Lily, Mrs. Potter]"
4,James Potter,/wiki/James_Potter_I,"[Prongs, James]"
...,...,...,...
702,Hugo Weasley,/wiki/Hugo_Granger-Weasley,[Hugo]
703,Scorpius Malfoy,/wiki/Scorpius_Malfoy,"[Scorpius, Malfoy the Unanxious, Scorpius the ..."
704,Victoire Weasley,/wiki/Victoire_Weasley,[Victoire]
705,Astoria Greengrass,/wiki/Astoria_Malfoy,[Astoria]


In [60]:
NnL.to_csv(r'Temp/CharacterWikis.csv',index=False)